## Import Libraries

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [4]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [21]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [22]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Split Data for Training and Validation


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size = 0.2, random_state=42)

## The training and validation set size

In [9]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [10]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [11]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [12]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [14]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [32]:
def encode_numerical_feature(column, name, dataset):

    norm = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = norm(column)

    return encoded_column

## Creating Tensor object from the column

In [18]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

## Encode columns based on their data type

In [20]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [23]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [28]:
x = keras.layers.Dense(64, activation = 'relu')(all_features)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [29]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [33]:
model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5947 - accuracy: 0.6501 - val_loss: 0.8083 - val_accuracy: 0.5359
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5838 - accuracy: 0.6528 - val_loss: 0.7850 - val_accuracy: 0.5028
Epoch 3/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5598 - accuracy: 0.6653 - val_loss: 0.8033 - val_accuracy: 0.4917
Epoch 4/300
37/37 [==============================] - 0s 4ms/step - loss: 0.6148 - accuracy: 0.6570 - val_loss: 0.7722 - val_accuracy: 0.5193
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5822 - accuracy: 0.6750 - val_loss: 0.8123 - val_accuracy: 0.4917
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5826 - accuracy: 0.6625 - val_loss: 0.8066 - val_accuracy: 0.5083
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6085 - accuracy: 0.6266 - val_loss: 0.7695 - val_accuracy: 0.5083
Epoch 8/300
3

Epoch 59/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5698 - accuracy: 0.6653 - val_loss: 0.8135 - val_accuracy: 0.5249
Epoch 60/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5892 - accuracy: 0.6556 - val_loss: 0.8302 - val_accuracy: 0.5304
Epoch 61/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5663 - accuracy: 0.6736 - val_loss: 0.8328 - val_accuracy: 0.5083
Epoch 62/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5794 - accuracy: 0.6584 - val_loss: 0.8336 - val_accuracy: 0.4972
Epoch 63/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5843 - accuracy: 0.6694 - val_loss: 0.8294 - val_accuracy: 0.5249
Epoch 64/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5840 - accuracy: 0.6598 - val_loss: 0.7893 - val_accuracy: 0.5304
Epoch 65/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5876 - accuracy: 0.6432 - val_loss: 0.7805 - val_accuracy: 0.5359
Epoch 

37/37 [==============================] - 0s 3ms/step - loss: 0.5705 - accuracy: 0.6542 - val_loss: 0.8173 - val_accuracy: 0.5414
Epoch 117/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5891 - accuracy: 0.6542 - val_loss: 0.7690 - val_accuracy: 0.5359
Epoch 118/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5810 - accuracy: 0.6625 - val_loss: 0.8396 - val_accuracy: 0.5359
Epoch 119/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5718 - accuracy: 0.6680 - val_loss: 0.8568 - val_accuracy: 0.5304
Epoch 120/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5696 - accuracy: 0.6680 - val_loss: 0.8744 - val_accuracy: 0.4862
Epoch 121/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5874 - accuracy: 0.6653 - val_loss: 0.8707 - val_accuracy: 0.5028
Epoch 122/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5922 - accuracy: 0.6279 - val_loss: 0.8143 - val_accuracy: 0.5249
Epoch 123/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5793 - accuracy: 0.6722 - val_loss: 0.9059 - val_accuracy: 0.5083
Epoch 174/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5648 - accuracy: 0.6708 - val_loss: 0.8658 - val_accuracy: 0.5525
Epoch 175/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5695 - accuracy: 0.6653 - val_loss: 0.8075 - val_accuracy: 0.5525
Epoch 176/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5514 - accuracy: 0.6763 - val_loss: 0.8607 - val_accuracy: 0.5359
Epoch 177/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5743 - accuracy: 0.6708 - val_loss: 0.8662 - val_accuracy: 0.5470
Epoch 178/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5757 - accuracy: 0.6598 - val_loss: 0.8369 - val_accuracy: 0.5635
Epoch 179/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5722 - accuracy: 0.6708 - val_loss: 0.8328 - val_accuracy: 0.5635
Epoch 180/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5655 - accuracy: 0.6639 - val_loss: 0.9185 - val_accuracy: 0.5304
Epoch 231/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5668 - accuracy: 0.6722 - val_loss: 0.8605 - val_accuracy: 0.5580
Epoch 232/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5516 - accuracy: 0.6750 - val_loss: 0.8551 - val_accuracy: 0.5525
Epoch 233/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5808 - accuracy: 0.6805 - val_loss: 0.8857 - val_accuracy: 0.5304
Epoch 234/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5724 - accuracy: 0.6846 - val_loss: 0.8645 - val_accuracy: 0.5691
Epoch 235/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5720 - accuracy: 0.6515 - val_loss: 0.8893 - val_accuracy: 0.5470
Epoch 236/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5691 - accuracy: 0.6708 - val_loss: 0.8817 - val_accuracy: 0.5746
Epoch 237/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5628 - accuracy: 0.6736 - val_loss: 0.8826 - val_accuracy: 0.5691
Epoch 288/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5705 - accuracy: 0.6846 - val_loss: 0.9005 - val_accuracy: 0.5525
Epoch 289/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5786 - accuracy: 0.6777 - val_loss: 0.9138 - val_accuracy: 0.5525
Epoch 290/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5830 - accuracy: 0.6473 - val_loss: 0.8760 - val_accuracy: 0.5691
Epoch 291/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5630 - accuracy: 0.6777 - val_loss: 0.8930 - val_accuracy: 0.5856
Epoch 292/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5760 - accuracy: 0.6501 - val_loss: 0.8855 - val_accuracy: 0.5414
Epoch 293/300
37/37 [==============================] - 0s 4ms/step - loss: 0.5600 - accuracy: 0.6625 - val_loss: 0.9792 - val_accuracy: 0.5193
Epoch 294/300